In [13]:
import importlib
# cim_profile = 'cimhub_2023'
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

In [14]:
from cimgraph.databases import ConnectionParameters
from cimgraph.databases import RDFlibConnection
from cimgraph.models import FeederModel
from cimgraph.models import NodeBreakerModel
from cimgraph.models import BusBranchModel
from test_get_all_edges import test_get_all_edges
import cimgraph.utils as utils

In [15]:
# cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
# params = ConnectionParameters(filename="./test_models/ieee13.xml", cim_profile='cimhub_2023', iec61970_301=8)
params = ConnectionParameters(filename="./test_models/IEEE123.xml", cim_profile='rc4_2021', iec61970_301=7)
# params = ConnectionParameters(filename=None, cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

# feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder_mrid = "_C1C3E687-6FFD-C753-582B-632A27E28507" #123 node
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)
# utils.get_all_data(network)
# utils.write_xml(network, "./test_output/ieee13test.xml")
# rdf.libgraph.serialize(destination='./test_output/test13rdf.xml', format='xml')

In [34]:
network.get_all_edges(cim.PowerTransformer)
network.get_all_edges(cim.TransformerTank)
network.get_all_edges(cim.PowerTransformerEnd)
for tank in network.graph[cim.PowerTransformer].values():
    print("end0", tank.PowerTransformerEnd[0].Terminal.mRID)
    # print("end1", tank.PowerTransformerEnd[1].Terminal.mRID)

IndexError: list index out of range

In [38]:
tank.name

'reg3'

In [7]:
sub = cim.Substation(name = 'new_sub', mRID='test_mrid_1234')
network.add_to_graph(sub)
network.pprint(cim.Substation)

{
    "test_mrid_1234": {
        "mRID": "test_mrid_1234",
        "name": "new_sub"
    }
}


In [ ]:
from cimgraph.utils.write_csv import write_csv

write_csv(network, './test_output')

In [ ]:
## Round-trip test
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_output/ieee13test.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"
feeder = cim.Feeder(mRID = feeder_mrid)
network = FeederModel(connection=rdf, container=feeder, distributed=False)
test_get_all_edges(network, cim)
utils.get_all_data(network)
utils.write_xml(network, "./test_output/ieee13test2.xml")
rdf.libgraph.serialize(destination='./test_output/test13rdf_2.xml', format='xml')

In [ ]:
## Distributed model test

network = FeederModel(connection=rdf, container=feeder, distributed=True)
for switch_area in network.distributed_areas:
    test_get_all_edges(switch_area, cim)
    for sec_area in switch_area.distributed_areas:
        test_get_all_edges(sec_area, cim)


In [ ]:
cim_profile = 'cimhub_2023'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_models/IEEE118_CIM.xml", cim_profile='cimhub_2023', iec61970_301=8)
rdf = RDFlibConnection(params)

feeder_mrid = "1783D2A8-1204-4781-A0B4-7A73A2FA6038" #IEEE 118 Bus"
feeder = cim.ConnectivityNodeContainer(mRID = feeder_mrid)
network = BusBranchModel(connection=rdf, container=feeder, distributed=False)
test_get_all_edges(network, cim)

In [ ]:
cim_profile = 'rc4_2021'
cim = importlib.import_module('cimgraph.data_profile.' + cim_profile)

# RDFLib File Reader Connection
params = ConnectionParameters(filename="./test_models/maple10nodebreaker.xml", cim_profile=cim_profile, iec61970_301=7)
rdf = RDFlibConnection(params)

feeder_mrid = "_EE4C60AE-550D-4599-92F4-022DF3118B3C"
geo = cim.GeographicalRegion(mRID = feeder_mrid)
network = NodeBreakerModel(connection=rdf, container=geo, distributed=True) 

for sr_area in network.distributed_areas[cim.SubGeographicalRegion].values():
    print("subregion", sr_area.container.name)
    test_get_all_edges(sr_area, cim)
    for sub_area in sr_area.distributed_areas[cim.Substation].values():
        print("substation", sub_area.container.name)
        test_get_all_edges(sub_area, cim)
        for vl_area in sub_area.distributed_areas[cim.VoltageLevel].values():
            print("voltage level", vl_area.container.name)
            test_get_all_edges(vl_area, cim)
        for feeder in sub_area.distributed_areas[cim.Feeder].values():
            print("feeder", feeder.container.name, feeder.container.mRID)
            test_get_all_edges(feeder, cim)



In [ ]:
network = NodeBreakerModel(connection=rdf, container=geo, distributed=False)

utils.get_all_data(network)